In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
import os
import math
%matplotlib inline

def grayscale(img):
    
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
   
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
   
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
   
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
 
    return cv2.addWeighted(initial_img, α, img, β, λ)

def average_line(myimage):
    
    myimage2 = grayscale(myimage)
    lt=140;
    ht=200;
    vertix_points = np.array([ [(0,540),(400,330),(560,330),(960,540)]], dtype=np.int32)

    myimage2=gaussian_blur(myimage2,3)
    canny_image=canny(myimage2,lt,ht)
    masked_image = region_of_interest(canny_image,vertix_points)


    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 8     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 8 #minimum number of pixels making up a line
    max_line_gap = 5 

    img_hough = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
    
    line_img=np.zeros_like(myimage) 
    
    a_left= np.array([])
    b_left= np.array([])
    a_right= np.array([])
    b_right= np.array([])
   
    my_lines = cv2.HoughLinesP(masked_image, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    for line in my_lines:
        for x1,y1,x2,y2 in line:
            if(0.8>((y2-y1)/(x2-x1))>0.4 and x1>480):
                a_right=np.append(a_right, np.polyfit([x1,x2],[y1,y2],1)[0])
                b_right=np.append(b_right, np.polyfit([x1,x2],[y1,y2],1)[1])
            elif(-0.8<((y2-y1)/(x2-x1))<-0.4 and x1<480):
                a_left=np.append(a_left, np.polyfit([x1,x2],[y1,y2],1)[0])
                b_left=np.append(b_left, np.polyfit([x1,x2],[y1,y2],1)[1])
    a_right_avg=np.mean(a_right)
    b_right_avg=np.median(b_right)
    a_left_avg=np.mean(a_left)
    b_left_avg=np.mean(b_left)
    
    x1_right=int((540-b_right_avg)/a_right_avg)
    x2_right=int((330-b_right_avg)/a_right_avg)
    x1_left=int((540-b_left_avg)/a_left_avg)
    x2_left=int((330-b_left_avg)/a_left_avg)
     
    cv2.line(line_img, (x1_right,540), (x2_right,330),[255, 0, 0], 8)
    cv2.line(line_img, (x1_left,540), (x2_left,330),[255, 0, 0],8)
    
    
    return line_img


test_images = os.listdir("test_images/")

def process_image(image):
   
    top_img = average_line(image)
    final_image = weighted_img(top_img,image, α=0.8, β=1., λ=0.)
    return final_image

# TODO: Build your pipeline that will draw lane lines on the test_images

myimage = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')

for test_image in test_images:
    out_image = mpimg.imread("test_images/{}".format(test_image))
    plt.imsave("test_images_output/{}".format(test_image), process_image(out_image))

#########################################

white_output = 'test_videos_output/solidWhiteRight.mp4'

clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

TypeError: draw_lines() takes 1 positional argument but 2 were given